In [1]:
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
import datetime
import os

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df_publishOrders = (
    spark
    .read
    .parquet(f"./jupyter_workspace/datalake/enriched/publish_orders/*")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_publishProduct = (
    spark
    .read
    .parquet(f"./jupyter_workspace/datalake/enriched/publish_product/*")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_publishOrders.filter(F.col('ProductID').isin('709')).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------+--------+---------+---------+-----------------+----------+----------+---------------+--------------+----------+-------------+-----------------+----------------------+----------------------+
|SalesOrderID|SalesOrderDetailID|OrderQty|ProductID|UnitPrice|UnitPriceDiscount|OrderDate |ShipDate  |OnlineOrderFlag|AccountNumber |CustomerID|SalesPersonID|TotalOrderFreight|LeadTimeInBusinessDays|TotalLineExtendedPrice|
+------------+------------------+--------+---------+---------+-----------------+----------+----------+---------------+--------------+----------+-------------+-----------------+----------------------+----------------------+
|43659       |10                |6       |709      |5.7      |0.0              |2021-05-31|2021-06-07|false          |10-4020-000676|29825     |279          |616.0984         |6                     |34.199997             |
|43665       |69                |6       |709      |5.7      |0.0              |2021-05-31|2021-06-07|false 

In [5]:
df_publishProduct.filter(F.col('ProductID').isin('709')).show(truncate=False, n=5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------------+-------------+--------+-----+----------------+------------+------------+---------+----+-------------------+------+---------------------+-------------------+----------------------+
|ProductID|ProductDesc           |ProductNumber|MakeFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|Weight|WeightUnitMeasureCode|ProductCategoryName|ProductSubCategoryName|
+---------+----------------------+-------------+--------+-----+----------------+------------+------------+---------+----+-------------------+------+---------------------+-------------------+----------------------+
|709      |Mountain Bike Socks, M|SO-B909-M    |false   |White|4               |3           |3.3963      |9.5      |M   |null               |null  |null                 |Clothing           |Socks                 |
+---------+----------------------+-------------+--------+-----+----------------+------------+------------+---------+----+-------------------+---

## Which color generated the highest revenue each year? 

In [6]:
window_spec = Window.partitionBy("year_OrderDate").orderBy(F.col("HighestRevenue_Color_Year").desc())

(
    df_publishOrders
    .join(df_publishProduct, df_publishProduct.ProductID == df_publishOrders.ProductID, how='left')
    .groupBy(
        df_publishProduct.Color,
        F.year(df_publishOrders.OrderDate).alias('year_OrderDate')
    )
    .agg(
        F.sum(df_publishOrders.TotalLineExtendedPrice).alias('HighestRevenue_Color_Year')
    )
    .withColumn("rn", F.row_number().over(window_spec)).filter(F.col("rn") == 1).drop("rn")
    .orderBy(F.col('year_OrderDate').desc())
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------+-------------------------+
| Color|year_OrderDate|HighestRevenue_Color_Year|
+------+--------------+-------------------------+
|Yellow|          2024|        6480746.113510132|
| Black|          2023|     1.5047694391397476E7|
| Black|          2022|     1.4005242933278084E7|
|   Red|          2021|        6019614.002418518|
+------+--------------+-------------------------+

## What is the average LeadTimeInBusinessDays by ProductCategoryName? 

In [7]:
(
    df_publishOrders
    .join(df_publishProduct, df_publishProduct.ProductID == df_publishOrders.ProductID, how='left')    
    .groupBy(df_publishProduct.ProductCategoryName)
    .agg(F.avg(df_publishOrders.LeadTimeInBusinessDays).alias('avg_LeadTimeInBusinessDays'))
).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------------+
|ProductCategoryName|avg_LeadTimeInBusinessDays|
+-------------------+--------------------------+
|              Bikes|         5.667897567632656|
|               null|         5.717621086432968|
|           Clothing|         5.709380234505863|
|        Accessories|         5.702787804316105|
|         Components|         5.667113624438874|
+-------------------+--------------------------+